In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


In [2]:
import csv
import os
import json
import pandas as pd
import plotly.express as px
import numpy as np



In [3]:
from app.flipside.models import Flipside
from app.oracles.models import Oracle
from app.constants import (
    oracle_feeds_lookup, oracle_feeds, 
    phase_windows, timestamp_windows,
    filenames_filtered, filenames
    )
from app.aave.models import AaveBalancePlotter
from app.combo.models import Combo
from app.swaps.models import DexSwaps
from app.visuals.plotly_helper import combined_action_layout, combined_action_layout_time

In [4]:
def fig_helper(fig, a=True, b=True, c=True, d=False):
    if a:
        fig.add_vrect(x0=timestamp_windows['michael_ramp_up'], x1=timestamp_windows['michael_ramp_stop'], 
                annotation_text='M: Setup', annotation_position="top left",
                fillcolor='blue', opacity=0.25, line_width=0)
    if b:
        fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_attack_start'], 
                annotation_text='A: Setup', annotation_position="top left",
                fillcolor='orange', opacity=0.25, line_width=0)
    if c:
        fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_attack_end'], 
                    annotation_text='Attack', annotation_position="top left",
                    fillcolor='red', opacity=0.25, line_width=0)
    if d:
        fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
                    annotation_text='A: Liq', annotation_position="top left",
                    fillcolor='purple', opacity=0.25, line_width=0)
    return fig


In [ ]:
fs = Flipside(filenames)

In [6]:
swaps = DexSwaps(fs)
volume_hourly = pd.DataFrame(swaps.hourly)
volume_daily = pd.DataFrame(swaps.daily)


In [ ]:
aave = AaveBalancePlotter(fs)
aave.set_start_values(
    'CRV',
    186012307.3451032,
    62424460.59351881,
    15962578
)
aave.iterate_through_blocks()

In [8]:
df = aave.balance_tracker
# t = fs.dataframes['aave_borrows']


In [ ]:

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(
    title=f"CRV Utilization",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig = fig.add_trace(
    go.Line(
        x=df.BLOCK_TIMESTAMP,
        y=df.TOTAL_COLLATERAL_BALANCE,
        name="CRV Collateral",
        mode='lines',
        line = dict(color='blue', width=2),
        line_shape='hv',
    ),
    secondary_y=False
)

fig = fig.add_trace(
    go.Line(
        x=df.BLOCK_TIMESTAMP,
        y=df.TOTAL_DEBT_BALANCE,
        name="CRV Debt", 
        line = dict(color='red', width=2),
        line_shape='hv',
        ),
    secondary_y=False
)

fig = fig.add_trace(
    go.Line(
        x=df.BLOCK_TIMESTAMP,
        y=df.UTILIZATION,
        name="CRV Utilization",
        mode='lines',
        line = dict(color='black', width=2, dash='dot'),
        line_shape='hv'
    ),
    secondary_y=True
)
fig = fig_helper(fig)

fig.show()


In [ ]:
fs2 = Flipside(filenames_filtered)

In [11]:
aave2 = AaveBalancePlotter(fs2)
aave2.set_start_values(
    'CRV',
    186012307.3451032,
    62424460.59351881,
    15962578
)
aave2.iterate_through_blocks()


In [12]:
oracles = Oracle(fs, oracle_feeds_lookup)

In [ ]:
target_address = '0x7a16ff8270133f063aab6c9977183d9e72835428'

combo_m = Combo(aave2, oracles, target_address)
combo_m.process_value(15962578 - 10, 16027236 +7200)
combo_m.process_sums()

target_address = '0x57e04786e231af3343562c062e0d058f25dace9e'
combo_a = Combo(aave2, oracles, target_address)
combo_a.process_value(15962578 - 10, 16027236 +7200)
combo_a.process_sums()

In [14]:
df_combo_m = pd.DataFrame(combo_m.records['CRV'])

df_combo_a = pd.DataFrame(combo_a.records['CRV'])


In [ ]:
fig2 = fig

fig2 = fig2.add_trace(
    go.Line(
        x=df_combo_m.block_timestamp,
        y=df_combo_m.c_balance,
        name="Michael CRV Collateral",
        mode='lines',
        line = dict(color='blue', width=2, dash='dash'),
        line_shape='hv'
    ),
    secondary_y=False
)

fig2 = fig2.add_trace(
    go.Line(
        x=df_combo_a.block_timestamp,
        y=df_combo_a.d_balance,
        name="Avi CRV Debt",
        mode='lines',
        line = dict(color='red', width=2, dash='dash'),
        line_shape='hv'
    ),
    secondary_y=False
)
fig2.show()

In [ ]:
fig2.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liq 1', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig2.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='Liq 2', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig2.show()

In [ ]:

fig5 = make_subplots(specs=[[{"secondary_y": True}]])
fig5.update_layout(
    title=f"CRV Collateral vs Volume",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig5 = fig5.add_trace(
    go.Line(
        x=df.BLOCK_TIMESTAMP,
        y=df.TOTAL_COLLATERAL_BALANCE,
        name="CRV Collateral",
        mode='lines',
        line = dict(color='blue', width=2),
        line_shape='hv',
    ),
    secondary_y=False
)

# fig6 = fig6.add_trace(
#     go.Line(
#         x=df.BLOCK_TIMESTAMP,
#         y=df.TOTAL_DEBT_BALANCE,
#         name="CRV Debt", 
#         line = dict(color='red', width=2),
#         line_shape='hv',
#         ),
#     secondary_y=False
# )

# fig5 = fig5.add_trace(
#     go.Line(
#         x=df.BLOCK_TIMESTAMP,
#         y=df.UTILIZATION,
#         name="CRV Utilization",
#         mode='lines',
#         line = dict(color='black', width=2, dash='dot'),
#         line_shape='hv'
#     ),
#     secondary_y=True
# )


fig5 = fig5.add_trace(
    go.Line(
        x=df_combo_m.block_timestamp,
        y=df_combo_m.c_balance,
        name="Michael CRV Collateral",
        mode='lines',
        line = dict(color='blue', width=2, dash='dash'),
        line_shape='hv'
    ),
    secondary_y=False
)

# fig5 = fig5.add_trace(
#     go.Line(
#         x=df_combo_a.block_timestamp,
#         y=df_combo_a.d_balance,
#         name="Avi CRV Debt",
#         mode='lines',
#         line = dict(color='red', width=2, dash='dash'),
#         line_shape='hv'
#     ),
#     secondary_y=False
# )
# fig5.show()

fig5 = fig5.add_trace(
    go.Bar(
        x=volume_daily.timestamp,
        y=volume_daily.total_volume,
        name="Daily Volume",
    ),
    secondary_y=False
)
fig5.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liquidations', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig5.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig5.show()

In [ ]:
144 / 5

In [ ]:

fig6 = make_subplots(specs=[[{"secondary_y": True}]])
fig6.update_layout(
    title=f"CRV Debt vs Volume",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

# fig6 = fig6.add_trace(
#     go.Line(
#         x=df.BLOCK_TIMESTAMP,
#         y=df.TOTAL_COLLATERAL_BALANCE,
#         name="CRV Collateral",
#         mode='lines',
#         line = dict(color='blue', width=2),
#         line_shape='hv',
#     ),
#     secondary_y=False
# )

fig6 = fig6.add_trace(
    go.Line(
        x=df.BLOCK_TIMESTAMP,
        y=df.TOTAL_DEBT_BALANCE,
        name="CRV Debt", 
        line = dict(color='red', width=2),
        line_shape='hv',
        ),
    secondary_y=False
)

# fig6 = fig6.add_trace(
#     go.Line(
#         x=df.BLOCK_TIMESTAMP,
#         y=df.UTILIZATION,
#         name="CRV Utilization",
#         mode='lines',
#         line = dict(color='black', width=2, dash='dot'),
#         line_shape='hv'
#     ),
#     secondary_y=True
# )


# fig6 = fig6.add_trace(
#     go.Line(
#         x=df_combo_m.block_timestamp,
#         y=df_combo_m.c_balance,
#         name="Michael CRV Collateral",
#         mode='lines',
#         line = dict(color='blue', width=2, dash='dash'),
#         line_shape='hv'
#     ),
#     secondary_y=False
# )

fig6 = fig6.add_trace(
    go.Line(
        x=df_combo_a.block_timestamp,
        y=df_combo_a.d_balance,
        name="Avi CRV Debt",
        mode='lines',
        line = dict(color='red', width=2, dash='dash'),
        line_shape='hv'
    ),
    secondary_y=False
)

fig6 = fig6.add_trace(
    go.Bar(
        x=volume_daily.timestamp,
        y=volume_daily.total_volume,
        name="Daily Volume",
    ),
    secondary_y=False
)
fig6.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liquidations', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig6.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig6.show()

In [ ]:
92 /144

# Morpho

In [ ]:
filenames_morpho = {
    # "aave_deposits": "v2_deposits_actors",
    # "aave_borrows": "v2_borrows_actors",
    # "aave_liquidations": "v2_liquidations_actors",
    "aave_repayments": "aave_repayments_filtered_morpho",
    "aave_withdraws": "aave_withdraws_filtered_morpho",
    # 'oracle_eth_crv': 'chainlink_crv-eth-download-oracles-results',
    # 'oracle_eth_dai': 'chainlink_dai-eth-download-oracles-results',
    "oracles": "chainlink_all_oracle_results3",
    "swaps": "ez_dex_swaps_filtered_crv" ,
    "transfers_to": 'acrv_token_transfers_morpho',
    'transfers_from': 'acrv_token_transfers_from_morpho'
}

In [ ]:

class AaveBalancePlotter():
    def __init__(self, fs):
        self.fs = fs
        # self.borrows = fs.dataframes['aave_borrows']
        self.repayments = fs.dataframes['aave_repayments']
        # self.deposits = fs.dataframes['aave_deposits']
        self.withdraws = fs.dataframes['aave_withdraws']
        # self.liquidations = fs.dataframes['aave_liquidations']
        # self.actions = [self.borrows, self.repayments, self.deposits, self.withdraws, self.liquidations]
        self.actions = [self.repayments, self.withdraws]
        self.balance_tracker = None
        self.total_symbol = "N/A"
        self.total_debt = 0
        self.total_collateral = 1
        self.total_start_block = 0

    def set_start_values(self, symbol, collateral, debt, start_block):
        self.total_symbol = symbol
        self.total_collateral = collateral
        self.total_debt = debt
        self.total_start_block = start_block
        
    
#     def find_block_boundries(self):
#         start_block = 1000000000000000
#         end_block = 0
#         for item in self.actions:
#             if item.iloc[-1]['BLOCK_NUMBER'] < comparator:
#                 start_block = item.iloc[-1]['BLOCK_NUMBER']
#             if item.iloc[1]['BLOCK_NUMBER'] > end_block:
#                 end_block = item.iloc[1]['BLOCK_NUMBER']
#         print(f"Start {start_block} | End {end_block}")
#         return [start_block, end_block]
    
    def iterate_helper(self, data, symbol, address, amount, is_positive=True):
        if not address in data:
            data[address] = {}
        if not symbol in data[address]:
            data[address][symbol] = 0
        if is_positive:
            # print(f"{data[address][symbol]} + {amount}")
            data[address][symbol] += float(amount)
            # print(f"= {data[address][symbol]}")
        else:
            # print(f"{data[address][symbol]} - {amount}")
            data[address][symbol] -= float(amount)
            # print(f"= {data[address][symbol]}")

        return data
    
    def fill_helper(self, data, address, symbol):
        if not address in data:
            data[address] = {}
        if not symbol in data[address]:
            data[address][symbol] = 0
        return data[address][symbol]

    def total_helper(self, symbol, amount, block, is_collateral = True, is_positive = True):
        if block >= self.total_start_block:
            if symbol == self.total_symbol:
                if is_collateral:
                    if is_positive:
                        self.total_collateral += amount
                    else:
                        self.total_collateral -= amount
                else:
                    if is_positive:
                        self.total_debt += amount
                    else:
                        self.total_debt -= amount

    def iterate_through_blocks(self):
        df = pd.concat(self.actions, ignore_index=True)
        df = df.sort_values("BLOCK_TIMESTAMP", axis = 0, ascending = True)
        c_tracker = {}
        d_tracker = {}
        collateral_column = []
        debt_column = []
        actor_column = []
        collateral_symbol = []
        debt_symbol = []
        total_collateral = []
        total_debt = []
        utilization = []
        block_number_column = []
        block_timestamp_column = []
        
        for index, row in df.iterrows():
#             print("_"*50)
#             print(index)
            # Deposits
            if not row['AAVE_VERSION']=="Aave V2":
                continue
            block_timestamp = row['BLOCK_TIMESTAMP']

            if not np.isnan(row['WITHDRAWN_TOKENS']):
                # print("withdraw")
                block_number = row['BLOCK_NUMBER']
                symbol = row['SYMBOL']
                amount = row['WITHDRAWN_TOKENS']
                actor_address = row['DEPOSITOR_ADDRESS']
                c_tracker = self.iterate_helper(c_tracker,
                                        symbol,
                                        actor_address,
                                        amount,
                                        False
                                        )
                
                collateral_column.append(c_tracker[actor_address][symbol])
                debt_column.append(self.fill_helper(d_tracker, actor_address, symbol))
                collateral_symbol.append(symbol)
                debt_symbol.append(symbol)
                self.total_helper(symbol, amount, block_number, True, False)
            # Borrowed
            # elif not np.isnan(row['BORROWED_TOKENS']):
            #     # print("borrow")
            #     block_number = row['BLOCK_NUMBER']
            #     amount = row['BORROWED_TOKENS']
            #     symbol = row['SYMBOL'] 
            #     actor_address = row['BORROWER_ADDRESS']
            #     d_tracker = self.iterate_helper(d_tracker,
            #                             symbol,
            #                             actor_address,
            #                             amount,
            #                             )
            #     debt_column.append(d_tracker[actor_address][symbol])
            #     collateral_column.append(self.fill_helper(c_tracker, actor_address, symbol))
            #     collateral_symbol.append(symbol)
            #     debt_symbol.append(symbol)
            #     self.total_helper(symbol, amount, block_number, False, True)

            # Repayment
            elif not np.isnan(row['REPAYED_TOKENS']):
                # print("repayment")
                block_number = row['BLOCK_NUMBER']
                amount = row['REPAYED_TOKENS']
                symbol = row['SYMBOL']
                actor_address = row['BORROWER']
                d_tracker = self.iterate_helper(d_tracker,
                                        symbol,
                                        actor_address,
                                        amount,
                                        False
                                        )
                debt_column.append(d_tracker[actor_address][symbol])
                collateral_column.append(self.fill_helper(c_tracker, actor_address, symbol))
                collateral_symbol.append(symbol)
                debt_symbol.append(symbol)
                self.total_helper(symbol, amount, block_number, False, False)
            # Liquidated
            # print(row['LIQUIDATED_AMOUNT'])
            # elif float(row['LIQUIDATED_AMOUNT']) > 0:
            #     # print("_"*50)
            #     # print(index)
            #     # print("liquidated")
            #     # print(row['LIQUIDATED_AMOUNT'])
            #     block_number = row['BLOCK_NUMBER']
            #     c_amount = row['LIQUIDATED_AMOUNT']
            #     c_symbol = row['COLLATERAL_TOKEN_SYMBOL']
            #     d_amount = row['DEBT_TO_COVER_AMOUNT']
            #     d_symbol = row['DEBT_TOKEN_SYMBOL']
            #     actor_address = row['BORROWER']

            #     # original_c = c_tracker[actor_address][c_symbol]
            #     # original_d = d_tracker[actor_address][d_symbol]


            #     c_tracker = self.iterate_helper(c_tracker,
            #                             c_symbol,
            #                             actor_address,
            #                             c_amount,
            #                             False
            #                             )
            #     collateral_column.append(c_tracker[actor_address][c_symbol])
            #     collateral_symbol.append(c_symbol)
            #     self.total_helper(c_symbol, c_amount, block_number, True, False)

            #     d_tracker = self.iterate_helper(d_tracker,
            #                             d_symbol,
            #                             actor_address,
            #                             d_amount,
            #                             False
            #                             )

            #     debt_column.append(d_tracker[actor_address][d_symbol])
            #     debt_symbol.append(d_symbol)
            #     self.total_helper(d_symbol, d_amount, block_number, False, False)
                # print('collateral')
                # print(original_c)
                # print(c_amount)
                # print(c_tracker[actor_address][c_symbol])
                # print(c_tracker[actor_address][c_symbol] == original_c - c_amount)
                # print("debt")
                # print(original_d)
                # print(d_amount)
                # print(d_tracker[actor_address][d_symbol])
                # print(d_tracker[actor_address][d_symbol] == original_d - d_amount)
            else:
                actor_address = "SKIP_RECORD"
                collateral_column.append(0)
                debt_column.append(0)
                collateral_symbol.append("SKIP")
                debt_symbol.append("SKIP")
                block_number = 0
                block_timestamp = "2022-10-06 06:45:14.000"	
                print(row)
            # if len(block_number_column)>0:
                # while block_number_column[-1] + 50 < block_number:
                #     collateral_column.append(collateral_column[-1])
                #     collateral_column.append(collateral_column[-1])
                #     collateral_column.append(collateral_column[-1])
                #     collateral_column.append(collateral_column[-1])
                #     collateral_column.append(collateral_column[-1])
                #     collateral_column.append(collateral_column[-1])
                #     collateral_column.append(collateral_column[-1])
            actor_column.append(actor_address)
            total_collateral.append(self.total_collateral)
            total_debt.append(self.total_debt)
            utilization.append(self.total_debt / self.total_collateral)
            block_number_column.append(block_number)
            block_timestamp_column.append(block_timestamp)
        
#         print(len(collateral_column))
#         print(len(debt_column))
        # Insert newly built columns
        data = {'ACTOR_ADDRESS': actor_column,
                'COLLATERAL_SYMBOL': collateral_symbol,
                'COLLATERAL_BALANCE': collateral_column,
                'DEBT_SYMBOL': debt_symbol,
                'DEBT_BALANCE': debt_column,
                'TOTAL_DEBT_BALANCE': total_debt,
                'TOTAL_COLLATERAL_BALANCE': total_collateral,
                'UTILIZATION': utilization,
                'BLOCK_NUMBER': block_number_column,
                'BLOCK_TIMESTAMP': block_timestamp_column,

        }
        df = pd.DataFrame(data)
        # df.insert(0, "ACTOR_ADDRESS", actor_column)
        # df.insert(1, "COLLATERAL_SYMBOL", collateral_symbol)
        # df.insert(2, "COLLATERAL_BALANCE", collateral_column)
        # df.insert(3, "DEBT_SYMBOL", debt_symbol)
        # df.insert(4, "DEBT_BALANCE", debt_column)
        # df.insert(5, "TOTAL_DEBT_BALANCE", total_debt)
        # df.insert(6, "TOTAL_COLLATERAL_BALANCE", total_collateral)
        # df.insert(7, "UTILIZATION", utilization)
        self.balance_tracker = df
                
    def get_combined_action_layout_inputs(self, target_address, window_start, window_end):
        deposits = self.deposits[self.deposits['DEPOSITOR_ADDRESS']== target_address]
        withdraws = self.withdraws[self.withdraws['DEPOSITOR_ADDRESS']== target_address]
        borrows = self.borrows[self.borrows['BORROWER_ADDRESS']== target_address]
        repayments = self.repayments[self.repayments['BORROWER']== target_address]
        liquidations = self.liquidations[self.liquidations['BORROWER']== target_address]

        deposits = self.fs.apply_window_df(deposits, window_start, window_end)
        withdraws = self.fs.apply_window_df(withdraws, window_start, window_end)
        borrows = self.fs.apply_window_df(borrows, window_start, window_end)
        repayments = self.fs.apply_window_df(repayments, window_start, window_end)
        liquidations = self.fs.apply_window_df(liquidations, window_start, window_end)

        return (deposits, withdraws, borrows, repayments, liquidations) 

In [ ]:
fs3 = Flipside(filenames_morpho)
aave3 = AaveBalancePlotter(fs3)
aave3.set_start_values(
    'CRV',
    186012307.3451032,
    62424460.59351881,
    15962578
)
aave3.iterate_through_blocks()




In [ ]:
target_address = '0x777777c9898d384f785ee44acfe945efdff5f3e0'.lower()

combo_morpho = Combo(aave3, oracles, target_address)
combo_morpho.process_value(15962578 - 10, 16027236 +7200)
combo_morpho.process_sums()

In [ ]:
df_combo_morpho = pd.DataFrame(combo_morpho.records['CRV'])
df_combo_morpho

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.update_layout(
    title=f"Morpho CRV Balances on AAVE (only withdraws and repyaments)",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig3 = fig3.add_trace(
    go.Scatter(
        x=df_combo_morpho.block_timestamp,
        y=df_combo_morpho.c_balance,
        name="Morpho CRV Collateral",
        mode='lines',
        marker_color="blue",
    ),
    secondary_y=False
)

fig3 = fig3.add_trace(
    go.Scatter(
        x=df_combo_morpho.block_timestamp,
        y=df_combo_morpho.d_balance,
        name="Morpho CRV Debt",
        mode='lines',
        marker_color="red",
    ),
    secondary_y=False
)

fig3.show()

In [ ]:
fig3.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liq 1', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig3.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='Liq 2', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig3.show()

In [ ]:
df_to = fs3.dataframes['transfers_to']
df_from = fs3.dataframes['transfers_from']

In [ ]:
df_to

In [ ]:


fig3 = fig3.add_trace(
    go.(
        x=df_to.BLOCK_TIMESTAMP,
        y=df_to.AMOUNT,
        name="aCRV Transfers To Morpho ",
        mode='markers',
        marker_color="green",
    ),
    secondary_y=False
)


fig3 = fig3.add_trace(
    go.Scatter(
        x=df_from.BLOCK_TIMESTAMP,
        y=df_from.AMOUNT,
        name="aCRV Transfers From Morpho ",
        mode='markers',
        marker_color="purple",
    ),
    secondary_y=False
)
fig3.show()

In [ ]:
14.78- 1.24

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.update_layout(
    title=f"Morpho aCRV Transfers",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig3 = fig3.add_trace(
    go.Scatter(
        x=df_to.BLOCK_TIMESTAMP,
        y=df_to.AMOUNT,
        name="aCRV Transfers To Morpho ",
        mode='markers',
        marker_color="green",
    ),
    secondary_y=False
)

fig3 = fig3.add_trace(
    go.Scatter(
        x=df_from.BLOCK_TIMESTAMP,
        y=df_from.AMOUNT,
        name="aCRV Transfers From Morpho ",
        mode='markers',
        marker_color="purple",
    ),
    secondary_y=False
)
fig3.show()

In [ ]:
fig3.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liq 1', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig3.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='Liq 2', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig3.show()

In [ ]:
volume_hourly = pd.DataFrame(swaps.hourly)
volume_daily = pd.DataFrame(swaps.daily)

In [ ]:
volume_daily

In [ ]:
# import plotly.express as px2
fig = px2.ecdf(df_to, x="BLOCK_TIMESTAMP", y="AMOUNT", markers=True, lines=False)
fig.show()

In [ ]:
df = aave.borrows[aave.borrows["BORROWER_ADDRESS"].isin(['0x777777c9898d384f785ee44acfe945efdff5f3e0'.lower()])]
df.keys()

In [ ]:
df = aave.deposits[aave.deposits["DEPOSITOR_ADDRESS"].isin(['0x777777c9898d384f785ee44acfe945efdff5f3e0'.lower()])]
df.keys()


In [ ]:
df = aave.borrows[aave.withdraws["DEPOSITOR_ADDRESS"].isin(['0x777777c9898d384f785ee44acfe945efdff5f3e0'.lower()])]
df

In [ ]:
df = aave.repayments[aave.repayments["BORROWER"].isin(['0x777777c9898d384f785ee44acfe945efdff5f3e0'.lower()])]
df